## Урок 2. Профилирование пользователей. Сегментация: unsupervised learning (clustering, LDA/ARTM), supervised (multi/binary classification)

In [18]:
!pip install gensim

  Using cached smart_open-5.2.1-py3-none-any.whl (58 kB)
  Attempting uninstall: Cython
    Found existing installation: Cython 0.29.21
    Uninstalling Cython-0.29.21:
      Successfully uninstalled Cython-0.29.21


In [1]:
import pandas as pd

In [4]:
news = pd.read_csv("articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


In [6]:
users = pd.read_csv("users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


**1. Получаем векторные представления новостей**

In [20]:
from gensim.test.utils import common_texts

In [21]:
from gensim.corpora.dictionary import Dictionary

In [25]:
!pip install razdel

  Using cached razdel-0.5.0-py3-none-any.whl (21 kB)


In [26]:
!pip install pymorphy2

  Using cached pymorphy2-0.9.1-py3-none-any.whl (55 kB)
  Using cached docopt-0.6.2.tar.gz (25 kB)
  Using cached DAWG_Python-0.7.2-py2.py3-none-any.whl (11 kB)
  Using cached pymorphy2_dicts_ru-2.4.417127.4579844-py2.py3-none-any.whl (8.2 MB)
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13709 sha256=85e64d6c4a5b45eddb9ed91adb27f206976a6197b41c4f49bedac3409d4932fe
  Stored in directory: c:\users\максим\appdata\local\pip\cache\wheels\56\ea\58\ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt


In [27]:
# предобработка текстов
import re
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from razdel import tokenize # https://github.com/natasha/razdel
import pymorphy2  

In [28]:
import nltk
nltk.download('stopwords')

stopword_ru = stopwords.words('russian')
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Максим\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [30]:
with open('stopwords.txt') as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [31]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens]

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [32]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

<ipython-input-31-7ee348d9b386>:15: FutureWarning: Possible nested set at position 39
  text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)


Wall time: 34.2 s


In [33]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

Wall time: 4min 17s


In [34]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]

In [35]:
common_dictionary[10]

'ватутин'

In [36]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus.
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

Wall time: 43.8 s


In [37]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model.lda")
lda.save(temp_file)

# Load a potentially pretrained model from disk.
lda = LdaModel.load(temp_file)

In [38]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc]

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'свой', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'провести', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'работа', 'сказать', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'мочь', 'играть', 'ещё', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'это', 'хороший']


[(10, 0.5724435), (14, 0.070725836), (15, 0.3372174)]

In [39]:
x=lda.show_topics(num_topics=25, num_words=7, formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))

topic_0: поверхность планета остров активность мировой доклад первый
topic_1: азербайджан задумка перехватить прохоров азербайджанец умышленно вилла
topic_2: ракета рак операция городской академия договор агентство
topic_3: северный космос белоруссия вирус офицер южный мышь
topic_4: газ погибнуть восток турция бомба греция иран
topic_5: автор метод вероятно опрос фаза предупредить напротив
topic_6: путин турецкий пресссекретарить песок учебный заведение жир
topic_7: пациент исследование кровь лечение зарубежный кожа проживать
topic_8: дональд лётчик донбасс инвестировать оплатить напомнить механический
topic_9: лёд золото кг корь кит золотой показ
topic_10: день nn пройти мероприятие сократиться сайт стать
topic_11: украина украинский россия российский объект который армия
topic_12: транспорт лауреат парк дед пиво денежный спасать
topic_13: это россия который страна мочь российский сша
topic_14: налог парламент медицина употребление мэй рт польский
topic_15: год компания рубль проект э

In [40]:
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)
    lda_tuple = lda[unseen_doc]
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [41]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,...,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.869826,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
1,4896,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.45421,0.111453,0.0,0.0,0.000000,0.0,0.0,0.000000
2,4897,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.337212,0.000000,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000
3,4898,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.331030,0.00000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.503899
4,4899,0.0,0.0,0.147907,0.0,0.0,0.0,0.0,0.0,0.0,...,0.094195,0.000000,0.00000,0.000000,0.0,0.0,0.330207,0.0,0.0,0.000000


In [42]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [43]:
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

In [44]:
doc_dict[293622]

array([0.        , 0.        , 0.03329752, 0.        , 0.        ,
       0.        , 0.03262457, 0.        , 0.        , 0.        ,
       0.24541806, 0.        , 0.        , 0.19285406, 0.        ,
       0.        , 0.        , 0.        , 0.01944386, 0.01900877,
       0.        , 0.        , 0.14466399, 0.        , 0.30237368])

**Домашнее задание**

Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)


Повторить п.2, но используя уже не медиану, а max
(опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.


Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

In [58]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, \
                                precision_recall_curve, confusion_matrix

In [59]:
user_articles_list = users['articles'].iloc[33]

def get_user_embedding(user_articles_list, metric=np.mean):
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    user_vector = metric(user_vector, 0)
    return user_vector

In [60]:
get_user_embedding(user_articles_list, np.median)

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.44526179, 0.        ,
       0.        , 0.04658673, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.17304518, 0.        , 0.1240224 ])

In [61]:
target = pd.read_csv("users_churn.csv")
print(target.shape)
target.head(3)

(8000, 2)


,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [62]:
# Функция расчета метрик
def get_metrics(metric):
    user_embeddings = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding(x, metric), 1)])
    user_embeddings.columns = [f'topic_{i}' for i in range(25)]
    user_embeddings['uid'] = users['uid'].values
    user_embeddings = user_embeddings[['uid']+[f'topic_{i}' for i in range(25)]]
    X = pd.merge(user_embeddings, target, 'left')

    X_train, X_test, y_train, y_test = train_test_split(X[[f'topic_{i}' for i in range(25)]], 
                                                        X['churn'], shuffle=True, train_size=0.7, random_state=42)

    logreg = LogisticRegression()
    logreg.fit(X_train, y_train)
    preds = logreg.predict_proba(X_test)[:, 1]

    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    roc_auc = roc_auc_score(y_test, preds)
    ix = np.argmax(fscore)
    
    return roc_auc, precision[ix], recall[ix], fscore[ix]

In [63]:
m = list(get_metrics(np.mean))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.914, precision = 0.475, recall = 0.798, fscore = 0.595


In [64]:
m = list(get_metrics(np.median))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.925, precision = 0.502, recall = 0.785, fscore = 0.612


In [65]:
m = list(get_metrics(np.max))
print(f'roc_auc = {m[0]:.3f}, precision = {m[1]:.3f}, recall = {m[2]:.3f}, fscore = {m[3]:.3f}')

roc_auc = 0.951, precision = 0.768, recall = 0.658, fscore = 0.709


In [66]:
metrics = {'mean': np.mean, 'median': np.median, 'max': np.max}
info = pd.DataFrame(columns=metrics.keys())

In [68]:
for metric, value in metrics.items():
    print(metric, value)
    roc_auc, precision, recall, fscore = get_metrics(value)
    info.loc[f'roc_auc', metric] = roc_auc
    info.loc[f'precision', metric] = precision
    info.loc[f'recall', metric] = recall
    info.loc[f'fscore', metric] = fscore
    
info

mean <function mean at 0x000001525F11DAF0>
median <function median at 0x000001525F258DC0>
max <function amax at 0x000001525F118D30>


,mean,median,max
roc_auc,0.914201,0.924635,0.951134
precision,0.474806,0.502083,0.768061
recall,0.798046,0.785016,0.65798
fscore,0.595383,0.612452,0.708772
